# Try Finetuned YOLO, ONNX Inference

Try inferencing with a finetuned YOLO checkpoint, using the ONNX runtime to serve the model

In [ ]:
%pip install -q opencv-python onnx onnxruntime torch requests
%pip list | awk '/opencv|onnx|onnxruntime|torch|requests/ {print $1}'

In [ ]:
# import libs
import os
from pathlib import PosixPath
import onnxruntime as rt
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
# declare global setting variables
HOME_DIR: str = os.getenv("HOME")
PRJ_PATH: str = f"{HOME_DIR}/camera-detection"
TEST_IMG: str = "aircraft_test.jpg"
LABEL_FILE: str = "labels.yaml"

# search onnx checkpoint
onnx_ckpt = max(PosixPath(PRJ_PATH).rglob("*.onnx"), key=os.path.getmtime)
print(f"Using YOLO Model Original Checkpoint at: ONNX - {onnx_ckpt}")

# load labels
import yaml
with open(LABEL_FILE, "r") as f:
    labels = yaml.safe_load(f).get("names")

## First thing, let's try inference locally by loading the ONNX checkpoint directly.

This will make sure the model works as expected in a controlled environment (e.g. a jupyter notebook)

In [ ]:
# load checkpoint from disk
onnx_model = rt.InferenceSession(onnx_ckpt, providers=rt.get_available_providers())

# get inputs and outputs of the onnx model
onnx_input = onnx_model.get_inputs()[0]
onnx_output = onnx_model.get_outputs()[0]
input_name = onnx_input.name
output_name = onnx_output.name

print(f"ONNX Model:\n Input: {input_name}, shape: {onnx_input.shape}\n Output: {output_name}, shape: {onnx_output.shape}")

In [ ]:
# datapoint visualization
def plot(img_rgb: torch.Tensor, inferenceOutput, confidences, classes, labels) -> None:
    """
        plot an image with its relative object bounding boxes in overlay
    """
    # permute channels and get image sizes
    img = img_rgb.permute(2, 1, 0)
    img_h, img_w, channels = img.shape

    # plot image
    plt.title(f"Dataset Point: {len(inferenceOutput)} objects")
    plt.imshow(img_rgb)

    # calculate bounding boxes
    axes = plt.gca()
    for i,o in enumerate(inferenceOutput):
        c = confidences[i]
        cl = classes[i]

        x1, y1, x2, y2 = o
        #label = object_classes[int(obj.cls)]
        print(f"Bounding Box: {x1}, {y1}, {x2}, {y2}")

        # add bounding box
        from matplotlib.patches import Rectangle
        axes.add_patch(Rectangle((x1,y1), x2,y2, color="white", fill=None))
        # add label
        ltext = f"{c} - {labels.get(cl)}"
        lpos = (x1, y1 - 10)
        axes.text(lpos[0], lpos[1], ltext, color="white", fontsize=12)

    # show datapoint
    plt.show()

In [ ]:
# load test image from filesystem
test_img = cv2.imread(TEST_IMG)
print(f"Loaded image {TEST_IMG} to memory. Image Size: {test_img.size}@({test_img.shape})")

# resize if necessary
onnx_input_img = test_img.copy()
N, C, W, H = onnx_input.shape
iH, iW, iC = test_img.shape
if (H != iH) or (W != iW):
    print(f"Resizing image to {W}x{H} - Current size is {iW}x{iH}")
    resized_img = cv2.resize(onnx_input_img, (W, H))
else:
    resized_img = onnx_input_img

# run inference
resized_img = np.transpose(resized_img, (2, 0, 1)).astype(np.float32) / 255.0
resized_img = np.expand_dims(resized_img, axis=0)
onnx_output = onnx_model.run(None, {input_name: resized_img})

In [ ]:
# display results
CONF_THRESHOLD = 0.55
predictions = onnx_output[0][0]
boxes_data = predictions[0:4, :]
scores_data = predictions[4:, :]

boxes, confidences, class_ids = [], [], []

for i in range(predictions.shape[1]):
    class_scores = 1 / (1 + np.exp(-scores_data[:, i]))
    max_score = np.max(class_scores)
    class_id = np.argmax(class_scores)
    cx, cy, w_box, h_box = boxes_data[:, i]

    if max_score > CONF_THRESHOLD:
        left = int((cx - w_box / 2) * iW / W)
        top = int((cy - h_box / 2) * iH / H)
        w_px = int(w_box * iW / W)
        h_px = int(h_box * iH / H)

        boxes.append([left, top, w_px, h_px])
        confidences.append(float(max_score))
        class_ids.append(class_id)

In [ ]:
print(boxes, confidences, class_ids)
plot(torch.from_numpy(test_img), boxes, confidences, class_ids, labels)

## Now let's call the remote inference service

In [ ]:
# endpoint
DEPLOYED_MODEL: str = "yolo-military-finetuned"
INFERENCE_SERVICE_ENDPOINT: str = "https://yolo-military-finetuned.yolo-finetune.svc.cluster.local"
INFER_URL: str = f"{INFERENCE_SERVICE_ENDPOINT}/v2/models/{DEPLOYED_MODEL}/infer"

# make request
def rest_request(infer_url, data):
    try:
        import requests
    except Exception as e:
        raise e

    json_data = {
        "inputs": [
            {
                "name": "images",
                "shape": [1, 3, 640, 640],
                "datatype": "FP32",
                "data": data,
            }
        ]
    }

    response = requests.post(infer_url, json=json_data, verify=True)
    print(response)
    response_dict = response.json()
    print(response_dict)
    return response_dict['outputs'][0]['data']

In [ ]:
# call the service
result = rest_request(INFER_URL, resized_img.tolist())
print(result)